In [1]:
%load_ext autoreload
%autoreload 2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
#import seaborn as sns
import sys
sys.path.append('../../')
from dataset import Dataset, SpikingDataset, RegSpikingDataset
from torch.utils.data.dataloader import DataLoader
from Loss import KDLoss
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
random.seed(1338)
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import copy
from pyESN import ESN
from scipy import interpolate
from gen_data import *
from tanh import tanh

import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader

# import slayer from lava-dl
import lava.lib.dl.slayer as slayer

import IPython.display as display
from matplotlib import animation


%load_ext autoreload

%autoreload 2

torch.__version__
# The coarse network structure is dicated by the Fashion MNIST dataset. 
dtype = torch.float

# Check whether a GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda:1')
    #device = torch.device("cuda")     
else:
    device = torch.device("cpu")
    
print(device)
tau_mem = 10e-3
tau_syn = 5e-3
time_step = 1e-3
alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))

weight_scale = 7*(1.0-beta) # this should give us some spikes to begin with

print("init done")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cpu
init done


In [2]:
silent = True
method = 'RLS'  # RLS; INV; INV+RLS
# N_total_frame = 17
N_total_frame = 94
N_sync_frame = 4
# SNR_list = np.arange(1,20,2)
SNR_list = [55]

# Dataset selection
folder_name = 'data/S2/'  # LOS_Near:S2, LOS_Far:S3, NLOS:S1
output_folder = 'data_outputs/S1'

if folder_name == 'data/S1/':  # NLOS
    delay = 0
    packet_num = 21
elif folder_name == 'data/S2/':  # LOS_Near
    delay = 1
    packet_num = 27 # correct
elif folder_name == 'data/S3/':  # LOS_Far
    delay = 1
    packet_num = 22 # 23
else:
    print("Undefined Dataset")
    exit(1)
    
window_size = 2
N_reservoir = 16
debug = False

ber_record = []
dfe_ber_record = []
LS_ber_record = []
comb_ber_record = []
sta_ber_record = []
tanh_lut = tanh(
    input_bit=8,
    dx_bit=8,
    slope_fmt=(10, 10),
    intercept_fmt=(19, 19),
    max=8,
    better_lut=True,
    verbose=False,
    plot=False)

SNR = SNR_list[0]
i = 1
rc = RC(silent, method, N_total_frame, N_sync_frame, SNR, delay, window_size, i,
        N_reservoir=16,
        spectral_radius=0.2,
        sparsity=0.4,
        noise=1e-6,
        lut_activation=False,  # True,
        tanh_lut=tanh_lut,
        input_scale=25,  #40, #50, # 25,
        reservoir_input_scale = 8,  #4,  #5,
        show_wout=False,
        output_folder= output_folder,
        debug=debug,
        use_fpga= None,
        sock= None,  # usock
        addr = None) # addr

train_input, train_label, test_input, test_label = rc.run()
RC_test_input = np.load('gt_test_input_1.npy')
RC_train_input = np.load('gt_train_input_1.npy')
RC_train_label = np.load('gt_train_label_1.npy')

print(RC_test_input.shape, test_input.shape)
print("test_input_diff: ", torch.nn.MSELoss()(torch.tensor(RC_test_input), torch.tensor(test_input)))

print("train_input_diff: ", torch.nn.MSELoss()(torch.tensor(RC_train_input), torch.tensor(train_input)))

print("train_label_diff: ", torch.nn.MSELoss()(torch.tensor(RC_train_label), torch.tensor(train_label)))

train_mean = np.mean(train_input)
train_std = np.std(train_input)

train_input = (train_input - train_mean) / train_std
test_input = (test_input - train_mean) / train_std

(7521, 4) (7521, 4)
test_input_diff:  tensor(4.3559e-10, dtype=torch.float64)
train_input_diff:  tensor(4.6755e-11, dtype=torch.float64)
train_label_diff:  tensor(0., dtype=torch.float64)


In [3]:
print(test_label[-26, 20 : 100])

[-0.70710677+0.70710677j  0.70710677-0.70710677j -0.70710677+0.70710677j
 -0.70710677-0.70710677j -0.70710677+0.70710677j -1.        +0.j
  0.70710677-0.70710677j  0.70710677-0.70710677j  0.70710677+0.70710677j
 -0.70710677-0.70710677j -0.70710677+0.70710677j  0.70710677+0.70710677j
  0.        +0.j          0.70710677+0.70710677j -0.70710677+0.70710677j
 -0.70710677-0.70710677j -0.70710677+0.70710677j  0.70710677-0.70710677j
  0.70710677+0.70710677j -1.        +0.j          0.70710677+0.70710677j
  0.70710677-0.70710677j  0.70710677-0.70710677j  0.70710677-0.70710677j
  0.70710677+0.70710677j -0.70710677-0.70710677j -0.70710677-0.70710677j
 -0.70710677+0.70710677j  0.70710677+0.70710677j -0.70710677+0.70710677j
  0.70710677-0.70710677j  0.70710677+0.70710677j -0.70710677+0.70710677j
  1.        +0.j          0.70710677+0.70710677j -0.70710677+0.70710677j
 -0.70710677+0.70710677j -0.70710677+0.70710677j -0.70710677+0.70710677j
  0.        +0.j          0.        +0.j          0.       

In [4]:
print(train_input.shape)
print(test_input.shape)
print(train_label.shape)
print(test_label.shape)

(7521, 4)
(7521, 4)
(7521, 2)
(89, 64)


In [5]:
import pandas as pd
import seaborn as sns


def pre_processing(train_input, test_input, train_label):
    idx_p = 10
    begin = -1 # N_total_frame * N_sync_frame

    train_label_df = pd.DataFrame(train_label, columns = ['L1','L2'])

    train_df = pd.DataFrame(train_input, columns = ['1','2', '3', '4'])
    train_df['L1_idx'] = train_df.index % idx_p
 
    train_label_df['L1_idx'] = train_label_df.index % idx_p

    # group by 
    mapping = train_label_df.loc[begin:, :].groupby(by='L1_idx').mean().reset_index().loc[:, ['L1', 'L2', 'L1_idx']] 
    
    train_df = pd.merge(train_df, mapping, how='left', on='L1_idx')

    train_df = pd.get_dummies(train_df, prefix=['L'], columns=['L1_idx'])

    train_df = train_df.loc[begin:, :]


    print(train_df.head(20))
    train_input = train_df.to_numpy()
    print(train_input.shape)

    test_df = pd.DataFrame(test_input, columns = ['1','2', '3', '4'])
    test_df['L1_idx'] = test_df.index % idx_p

    #group by
    test_df = test_df.merge(mapping, how = 'left', on='L1_idx')

    test_df = pd.get_dummies(test_df, prefix=['L'], columns=['L1_idx'])

    print(test_df.head())
    test_input = test_df.to_numpy()
    print(test_input.shape)
    
    return train_input, test_input

train_input, test_input = pre_processing(train_input, test_input, train_label)

           1         2         3         4        L1        L2  L_0  L_1  L_2  \
0   0.004553  0.004553 -1.888441  0.270612 -0.002946  0.002839    1    0    0   
1  -1.888441  0.270612  0.812097 -5.109579 -0.007376  0.005062    0    1    0   
2   0.812097 -5.109579  2.425456  1.353290 -0.005515 -0.000335    0    0    1   
3   2.425456  1.353290  0.278971  5.126545  0.005362 -0.004129    0    0    0   
4   0.278971  5.126545  0.012743  3.788194  0.005562 -0.000661    0    0    0   
5   0.012743  3.788194  1.354605  4.865612  0.003752  0.000764    0    0    0   
6   1.354605  4.865612  2.707468 -2.700189  0.004581  0.003351    0    0    0   
7   2.707468 -2.700189 -1.343801 -4.040178  0.003330  0.003798    0    0    0   
8  -1.343801 -4.040178 -0.263666  2.431761 -0.003775 -0.003522    0    0    0   
9  -0.263666  2.431761  5.128487 -1.344931 -0.009681 -0.004466    0    0    0   
10  5.128487 -1.344931 -1.625135 -2.413300 -0.002946  0.002839    1    0    0   
11 -1.625135 -2.413300 -5.12

In [5]:
nb_inputs  = 8
nb_hidden  = 96
nb_outputs = 2

time_step = 1e-3
nb_steps  = 100

batch_size = 128

import scipy.io

from dataset import Dataset, RegTorchSeasonalitySpikingDataset, RegSpikingDataset, RegTorchSpikingDataset
train_data = RegTorchSeasonalitySpikingDataset(train_input, train_label, nb_inputs, nb_steps)
test_data = RegTorchSeasonalitySpikingDataset(test_input, train_label, nb_inputs, nb_steps)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False, drop_last=False)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, drop_last=False)
print(len(train_data))
print(train_data[0][0].shape)
print(train_data[0][1].shape)
print(train_data[0][2].shape)
print(train_data[0][3].shape)

7521
(32, 100)
(1,)
(0,)
(2,)


In [9]:
class Network(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(Network, self).__init__()

       # neuron_params = {
       #         'threshold'     : 0.1,
       #         'current_decay' : 1,
       #         'voltage_decay' : 0.1,
       #         'requires_grad' : True,     
       #     }
        #neuron_params_drop = {**neuron_params, 'dropout' : slayer.neuron.Dropout(p=0.05),}
        neuron_params = {
                'threshold'     : 1.25,
                'current_decay' : 0.25,
                'voltage_decay' : 0.03,
                'tau_grad'      : 0.03,
                'scale_grad'    : 100,
                'requires_grad' : True,     
            }
        
        
        self.blocks = torch.nn.ModuleList([
                #slayer.block.cuba.Input(neuron_params),
                slayer.block.cuba.Recurrent(neuron_params, input_size, 64, weight_norm=True, delay=True),
                #slayer.block.cuba.Dense(neuron_params, 32, 64, weight_norm=True, delay=True),
                slayer.block.cuba.Dense(neuron_params, 64, 128, weight_norm=True, delay=True),
                slayer.block.cuba.Dense(neuron_params, 128, output_size, weight_norm=True),
                #slayer.block.sigma_delta.Dense(sdnn_dense_params, input_size, 64, weight_scale=2, weight_norm=True),
                #slayer.block.sigma_delta.Dense(sdnn_dense_params, 64, 128, weight_scale=2, weight_norm=True),
                #slayer.block.sigma_delta.Dense(sdnn_dense_params, 128, output_size, weight_scale=2, weight_norm=True)
                #slayer.block.cuba.Recurrent(cuba_params, 100, 50),
                #slayer.block.cuba.KWTA(cuba_params, 50, 50, num_winners=5)
            ])
    
    def forward(self, spike):
        for block in self.blocks:
            spike = block(spike)
        return spike

    def export_hdf5(self, filename):
        # network export to hdf5 format
        h = h5py.File(filename, 'w')
        layer = h.create_group('layer')
        for i, b in enumerate(self.blocks):
            b.export_hdf5(layer.create_group(f'{i}'))

class DNNNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()

        self.fc1 = nn.Linear(input_size, 16)
        self.fc2 = nn.Linear(16 + 1 + 12, 8)
        self.fc3 = nn.Linear(8, 2)
        #self.fc3 = nn.Linear(128, 128)
        #self.fc4 = nn.Linear(128, 2)
        self.act = nn.ReLU()

    def forward(self,x, x1, x2):
        x = x.flatten(start_dim=1)
        #x = torch.cat((x, x1), axis=1)
        x = self.fc1(x)
        x = self.act(x) 
        x = torch.cat((x, x1, x2), axis=1)
        x = self.fc2(x)
        x = self.act(x)
        x = self.fc3(x)
        #x = self.fc1(x)
        #x = self.act(x)
        #x = self.fc2(x)
        #x = self.act(x)
        #x = self.fc3(x)
        #x = self.act(x)
        #x = self.fc4(x)
        return x

In [10]:
net_snn = Network(nb_inputs * 4, 2).to(device)
net_dnn = DNNNetwork(2 * 100, 2).to(device)

for name, weight in net_snn.named_parameters():
    print(name, weight.shape)


blocks.0.neuron.current_decay torch.Size([1])
blocks.0.neuron.voltage_decay torch.Size([1])
blocks.0.delay.delay torch.Size([1])
blocks.0.input_synapse.weight_g torch.Size([64, 1, 1, 1, 1])
blocks.0.input_synapse.weight_v torch.Size([64, 32, 1, 1, 1])
blocks.0.recurrent_synapse.weight_g torch.Size([64, 1, 1, 1, 1])
blocks.0.recurrent_synapse.weight_v torch.Size([64, 64, 1, 1, 1])
blocks.1.neuron.current_decay torch.Size([1])
blocks.1.neuron.voltage_decay torch.Size([1])
blocks.1.delay.delay torch.Size([1])
blocks.1.synapse.weight_g torch.Size([128, 1, 1, 1, 1])
blocks.1.synapse.weight_v torch.Size([128, 64, 1, 1, 1])
blocks.2.neuron.current_decay torch.Size([1])
blocks.2.neuron.voltage_decay torch.Size([1])
blocks.2.synapse.weight_g torch.Size([2, 1, 1, 1, 1])
blocks.2.synapse.weight_v torch.Size([2, 128, 1, 1, 1])


In [9]:
RC_test_time = np.load('gt_time.npy')
RC_test_freq = np.load('gt_freq.npy')
RC_test_input = np.load('gt_test_input_1.npy')

def test(test_loader, net_snn, net_dnn):
    all_output = []
    inputs = []
    labels = []
    
    for input, input1, input2, target in test_loader:
        inputs.append(input)
        labels.append(target)
        target = torch.tensor(target).float()
        input = torch.tensor(input).float()
        input1 = torch.tensor(input1).float()
        input2 = torch.tensor(input2).float()
    
        output = net_snn(input)
        output = net_dnn(output, input1, input2).cpu().detach().numpy()
        all_output.append(output)
    
    all_output = np.concatenate(all_output, axis=0)
    print("test MSE: ", torch.nn.MSELoss()(torch.tensor(all_output), torch.tensor(RC_test_time)))
    ber = rc.my_test(all_output)
    print(ber)
    
test(test_loader, net_snn, net_dnn)

/tmp/ipykernel_533/3076915975.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target).float()
/tmp/ipykernel_533/3076915975.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input).float()
/tmp/ipykernel_533/3076915975.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input1 = torch.tensor(input1).float()
/tmp/ipykernel_533/3076915975.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_gra

test MSE:  tensor(0.3056, dtype=torch.float64)
[ 6  7  8  9 10 12 13 14 15 16 17 18 19 20 21 22 23 24 26 27 28 29 30 31
 33 34 35 36 37 38 40 41 42 43 44 45 46 47 48 49 50 51 52 54 55 56 57 58]
0.5304307116104869


In [10]:
RC_train_time = np.load("gt_train_pred.npy")
print("train MSE: ", torch.nn.MSELoss()(torch.tensor(RC_train_time), torch.tensor(train_label)))
def test_train(test_loader, net_snn, net_dnn):
    all_output = []
    inputs = []
    labels = []
    
    for input, input1, input2, target in test_loader:
        inputs.append(input)
        labels.append(target)
        target = torch.tensor(target).float()
        input = torch.tensor(input).float()
        input1 = torch.tensor(input1).float()
        input2 = torch.tensor(input2).float()
    
        output = net_snn(input)
        output = net_dnn(output, input1, input2).cpu().detach().numpy()
        all_output.append(output)
    
    all_output = np.concatenate(all_output, axis=0)
    print("train MSE: ", torch.nn.MSELoss()(torch.tensor(all_output), torch.tensor(RC_train_time)))

train MSE:  tensor(0.0033, dtype=torch.float64)


In [23]:
def train(trainloader, testloader, model, DNN_model, lr=2e-3, nb_epochs=10):
    #params = [w1,w2]
    params = list(model.parameters()) + list(DNN_model.parameters())
    optimizer = torch.optim.Adam(params, lr=lr, betas=(0.9,0.999))
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)
    
    #loss_fn = slayer.loss.SpikeTime(time_constant=2, filter_order=2, reduction='mean').to(device)
    loss_fn = torch.nn.MSELoss()
    #loss_fn = torch.nn.SmoothL1Loss()
    loss_hist = []
    DNN_model.train()
    
    
    for e in range(nb_epochs):
        print(e)
        local_loss = []
        for x_local, x1_local, x2_local, y_local in trainloader:
            x_local = x_local.float().to(device)
            x1_local = x1_local.float().to(device)
            x2_local = x2_local.float().to(device)
            y_local = y_local.float().to(device)
    
            #output = model(x_local)
            #output = output.flatten(start_dim=1)
            optimizer.zero_grad()
            output = model(x_local)
            output = DNN_model(output, x1_local, x2_local)
            loss_val = loss_fn(output, y_local) 
            loss_val.backward()
            #print("AAAA: ", DNN_model.fc2.weight)
            #print("BBBB: ", DNN_model.fc1.weight.grad)
            optimizer.step()
            local_loss.append(loss_val.item())
            
        
        #if e % 1 == 30 and e != 0:
        #    print("Training accuracy: %.3f"%(compute_ber(trainloader, net, "train")))
        #    print("Test accuracy: %.3f"%(compute_ber(testloader, net, name)))
        scheduler.step()
        mean_loss = np.mean(local_loss)
        print(mean_loss)
        #print("Epoch %i: loss=%.5f"%(e+1,mean_loss))
        test(testloader, model, DNN_model)
        test_train(trainloader, model, DNN_model)

        

In [11]:
#train(train_loader, test_loader, net_snn, net_dnn, lr=1e-3, nb_epochs=100)

In [25]:
RC_train_time = np.load("gt_train_pred.npy")
print("train MSE: ", torch.nn.MSELoss()(torch.tensor(RC_train_time), torch.tensor(train_label)))
def test_train_both(train_loader, net_snn, net_dnn):
    all_output = []
    inputs = []
    labels = []
    
    for input, input1, input2, target in train_loader:
        inputs.append(input)
        labels.append(target)
        target = torch.tensor(target).float()
        input = torch.tensor(input).float()
        input1 = torch.tensor(input1).float()
        input2 = torch.tensor(input2).float()
    
        output = net_snn(input)
        output = net_dnn(output, input1, input2).cpu().detach().numpy()
        all_output.append(output)
    
    all_output = np.concatenate(all_output, axis=0)
    print("train MSE RC: ", torch.nn.MSELoss()(torch.tensor(all_output), torch.tensor(RC_train_time)))
    print("train MSE GT: ", torch.nn.MSELoss()(torch.tensor(all_output), torch.tensor(train_label)))
    print("train MSE GT: ", torch.nn.MSELoss()(torch.tensor(train_label), torch.tensor(RC_train_time)))

#train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False, drop_last=False)
test_train_both(train_loader, net_snn, net_dnn)

train MSE:  tensor(0.0033, dtype=torch.float64)


/tmp/ipykernel_219/3403701246.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target).float()
/tmp/ipykernel_219/3403701246.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input).float()
/tmp/ipykernel_219/3403701246.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input1 = torch.tensor(input1).float()
/tmp/ipykernel_219/3403701246.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_gra

train MSE RC:  tensor(0.0051, dtype=torch.float64)
train MSE GT:  tensor(0.0036, dtype=torch.float64)
train MSE GT:  tensor(0.0033, dtype=torch.float64)


In [13]:
net_snn.export_hdf5('./net_snnaaaaaaaaa.net')
torch.save(net_dnn.state_dict(), './net_dnn.pth')

TypeError: Object dtype dtype('O') has no native HDF5 equivalent

In [40]:
temp_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False, drop_last=False)
def test_MSE(test_loader, net_snn, net_dnn):
    all_output = []
    inputs = []
    labels = []
    
    for input, input1, input2, target in test_loader:
        inputs.append(input)
    
        target = torch.tensor(target).float()
        input = torch.tensor(input).float()
        input1 = torch.tensor(input1).float()
        input2 = torch.tensor(input2).float()
    
        output = net_snn(input)
        output = net_dnn(output, input1, input2).cpu().detach().numpy()
        all_output.append(output)
        labels.append(target.cpu().detach().numpy())
    
    all_output = torch.tensor(np.concatenate(all_output, axis=0)).detach()
    labels = torch.tensor(np.concatenate(labels, axis=0)).detach()
    print(all_output.shape, labels.shape)
    loss_fn = torch.nn.MSELoss()
    
    print("all_loss: ", loss_fn(all_output, labels))
    print("no first 330 loss: ", loss_fn(all_output[330:, :], labels[330:, :]))

test_MSE(temp_loader, net_snn, net_dnn)

/tmp/ipykernel_98/229205384.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target).float()
/tmp/ipykernel_98/229205384.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input).float()
/tmp/ipykernel_98/229205384.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input1 = torch.tensor(input1).float()
/tmp/ipykernel_98/229205384.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True)

torch.Size([7521, 2]) torch.Size([7521, 2])
all_loss:  tensor(0.0024)
no first 330 loss:  tensor(0.0014)


In [41]:
all_output = []
inputs = []
labels = []
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, drop_last=False)
def test(test_loader, net_snn, net_dnn):
    all_output = []
    inputs = []
    labels = []
    
    for input, input1, input2, target in test_loader:
        inputs.append(input)
        labels.append(target)
        target = torch.tensor(target).float()
        input = torch.tensor(input).float()
        input1 = torch.tensor(input1).float()
        input2 = torch.tensor(input2).float()
    
        output = net_snn(input)
        output = net_dnn(output, input1, input2).cpu().detach().numpy()
        all_output.append(output)
    
    all_output = np.concatenate(all_output, axis=0)
    ber = rc.my_test(all_output)
    print(ber)
    
test(test_loader, net_snn, net_dnn)

/tmp/ipykernel_98/3202473027.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target).float()
/tmp/ipykernel_98/3202473027.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input).float()
/tmp/ipykernel_98/3202473027.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input1 = torch.tensor(input1).float()
/tmp/ipykernel_98/3202473027.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(T

0.18902153558052434


In [11]:
A1 = 8 * torch.ones((128, 8))
A2 = torch.ones((128, 1))
print(A1 + A2)

tensor([[9., 9., 9.,  ..., 9., 9., 9.],
        [9., 9., 9.,  ..., 9., 9., 9.],
        [9., 9., 9.,  ..., 9., 9., 9.],
        ...,
        [9., 9., 9.,  ..., 9., 9., 9.],
        [9., 9., 9.,  ..., 9., 9., 9.],
        [9., 9., 9.,  ..., 9., 9., 9.]])
